# TITLE

We downloaded the files from: <br>
accidentalidad.csv: http://medata.gov.co/dataset/accidentalidad 
<br>
AccidentalidadgeoreferenciadaYYYY:
<br>
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2019
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2018
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2017
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2016
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2015
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2014
<br>

To ingest about dataset we decided to use two different dataframes, ```df``` to aggregate the differents files by year and include the incident time wich is not included in the ```agg_df```, so we use these two dataframes to verify the integrity of the datasets and build one dataset with all the information we need.

In the following code we clean and merge the datasets:

In [102]:
import pandas as pd
from ast import literal_eval
from unidecode import unidecode
pd.options.display.max_columns = None

# agg_df: yearly file
agg_df = pd.DataFrame()
path = "./"
for  file in os.listdir(path):
    if (file.split(".")[-1] == "csv") and (file != 'accidentalidad.csv'):
        df_tmp = pd.read_csv(os.path.join(path, file))
        df_tmp.columns = [c.lower() for c in df_tmp.columns]
        df_tmp['file'] = file
        agg_df = pd.concat([agg_df, df_tmp])

In [103]:
agg_df.columns

Index(['x', 'y', 'objectid', 'radicado', 'fecha', 'hora', 'dia', 'periodo',
       'clase', 'direccion', 'direccion_enc', 'cbml', 'tipo_geocod',
       'gravedad', 'barrio', 'comuna', 'diseno', 'dia_nombre', 'mes',
       'mes_nombre', 'x_magnamed', 'y_magnamed', 'longitud', 'latitud', 'file',
       'shape'],
      dtype='object')

## How did you merge the two DataFrames (aggregated data from 2014-2019 and accidentalidad.csv)

In [104]:
# scr accidentalidad.csv: http://medata.gov.co/dataset/accidentalidad 
df =  pd.read_csv('./accidentalidad.csv', skiprows=1,header=None,sep=';')
df.loc[0:14]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,2019-06-04T20:44:15.587Z,NaN,A000010886,No Ubicada,0,Atropello,"[-75.7037762763, 6.22141524356]",3.0,2014.0,NaN,[u'incidentes'],Tramo 9 Kilometro 999,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,Sin Inf,Tramo de via,NaN,Con heridos,2014-03-12T10:30:00.000Z,1180000.00,820000.00,1.0,8626.0,1432623
1,2019-06-04T20:44:15.588Z,809,A000010843,Malla vial,CL 052 A 032 000 00000,Caída de Ocupante,"[-75.5534544709, 6.24349597268]",3.0,2014.0,Villa Hermosa,[u'incidentes'],CL 52 A CR 32,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,08,Tramo de via,Sucre,Con heridos,2014-03-12T10:00:00.000Z,1182393.81,836648.68,1.0,8628.0,1432625
2,2019-06-04T20:44:15.588Z,NaN,A000010821,No Ubicada,0,Caída de Ocupante,"[-75.7037762763, 6.22141524356]",3.0,2014.0,NaN,[u'incidentes'],CR 62 CL 84,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,Sin Inf,Tramo de via,NaN,Con heridos,2014-03-13T00:20:00.000Z,1180000.00,820000.00,1.0,8630.0,1432627
3,2019-06-04T20:44:15.588Z,514,A000010849,Malla vial,CR 073 B 091 000 00000,Atropello,"[-75.5792346017, 6.28690605219]",3.0,2014.0,Castilla,[u'incidentes'],CR 73 B CL 91,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,05,Tramo de via,Alfonso López,Con heridos,2014-03-12T23:00:00.000Z,1187204.11,833808.55,1.0,8632.0,1432630
4,2019-06-04T20:44:15.589Z,1621,A000010855,Malla vial,CR 053 030 A 000 00000,Otro,"[-75.5778859663, 6.23258248117]",3.0,2014.0,Belén,[u'incidentes'],CR 53 CL 30 A,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,16,Tramo de via,Cerro Nutibara,Con heridos,2014-03-13T04:50:00.000Z,1181194.15,833940.67,1.0,8639.0,1432637
5,2019-06-04T20:44:15.589Z,6098,A000010826,Malla vial,CR 104 064 000 00000,Caída de Ocupante,"[-75.6151738325, 6.281156378]",3.0,2014.0,Corregimiento de San Cristóbal,[u'incidentes'],CR 104 CL 64,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,60,Tramo de via,Área de Expansión Pajarito,Con heridos,2014-03-13T01:00:00.000Z,1186579.60,829828.47,1.0,8642.0,1432640
6,2019-06-04T20:44:15.589Z,1101,A000010917,Malla vial,CR 065 051 000 00000,Choque,"[-75.58085586, 6.25657541199]",3.0,2014.0,Laureles Estadio,[u'incidentes'],CR 65 CL 51,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,11,Tramo de via,Carlos E. Restrepo,Con heridos,2014-03-12T15:00:00.000Z,1183849.30,833619.47,1.0,8649.0,1432721
7,2019-06-04T20:44:15.589Z,1318,A000010841,Malla vial,CL 044 105 000 00000,Choque,"[-75.6200332483, 6.25681851817]",3.0,2014.0,San Javier,[u'incidentes'],CL 44 CR 105,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,13,Tramo de via,Antonio Nariño,Con heridos,2014-03-12T23:00:00.000Z,1183888.76,829282.64,1.0,8650.0,1432722
8,2019-06-04T20:44:15.590Z,307,A000010916,Malla vial aproximada: CR 45-67,CR 046 067 000 00000,Caída de Ocupante,"[-75.5555813857, 6.2631762147]",3.0,2014.0,Manrique,[u'incidentes'],CR 46 CL 67,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,03,Tramo de via,Manrique Central No. 2,Con heridos,2014-03-12T17:00:00.000Z,1184571.57,836419.35,1.0,8659.0,1432732
9,2019-06-04T20:44:15.590Z,1013,A000010992,Malla vial,CR 045 048 000 00000,Caída de Ocupante,"[-75.568553277, 6.24524859423]",3.0,2014.0,La Candelaria,[u'incidentes'],CR 45 CL 48,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,10,Tramo de via,Barrio Colón,Con heridos,2014-03-12T16:30:00.000Z,1182592.40,834977.79,1.0,8663.0,1432736


In [105]:
agg_df.shape, df.shape

((251899, 26), (287759, 24))

In [106]:
df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23],
           dtype='int64')

We get the column names from column 4 where the column 0 was equal to "1.0", then we coppied the column names and we assigned to the columns in the DataFrame (df) as follow:

In [107]:
from ast import literal_eval
[l.lower() for l in list(literal_eval(list(df.loc[df[0]=="1.0", 4][0:1])[0]).keys())]


['@timestamp',
 'cbml',
 'expediente',
 'tipo',
 'direccion encasillada',
 'clase_accidente',
 'location',
 'mes',
 'fecha_accidentes',
 'año',
 'comuna',
 'tags',
 'direccion',
 'host',
 'fingerprint',
 'path',
 'numcomuna',
 'diseño',
 'barrio',
 'gravedad_accidente',
 'fecha_accidente',
 'y',
 'x',
 '@version',
 'id',
 'nro_radicado']

We found values in column 0 unformatted so we procedure to drop them.

In [108]:
# dic = literal_eval(df[df[0]=="1.0"][4].values[0])
# print(dic)
# df.columns = [col.lower() for col in dic.keys()]
# print(df.columns)

cols = ["timestamp", "cbml", "expediente", "tipo_geocode", "direccion_enc", "clase", "x_long_y_lat", "mes", "periodo", "comuna", "incidente", "direccion", "operador", "path", "num_comuna", "diseno", "barrio", "gravedad", "fecha", "y_magnamed", "x_magnet", "version", "id", "radicado"]
df.columns = cols
df = df[df.timestamp != '1.0']
df[df.timestamp == '1.0']


,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado


In [109]:
agg_df.radicado.head()

0    1517771
1    1517787
2    1517740
3    1517747
4    1517993
Name: radicado, dtype: object

In [110]:
agg_df.info

<bound method DataFrame.info of                x         y  objectid     radicado                   fecha  \
0     -75.553148  6.291689    257034      1517771  2016/01/01 00:00:00+00   
1     -75.637704  6.275732    257035      1517787  2016/01/01 00:00:00+00   
2     -75.555730  6.305737    257037      1517740  2016/01/01 00:00:00+00   
3     -75.583366  6.293494    257038      1517747  2016/01/01 00:00:00+00   
4     -75.581203  6.195227    257039      1517993  2016/01/01 00:00:00+00   
...          ...       ...       ...          ...                     ...   
40343 -75.563465  6.253182    686841  1.65195e+06  2018/12/01 00:00:00+00   
40344 -75.574709  6.236105    686842    1.652e+06  2018/12/01 00:00:00+00   
40345 -75.599798  6.265247    686843  1.65192e+06  2018/12/01 00:00:00+00   
40346 -75.582006  6.215066    686844  1.65188e+06  2018/12/01 00:00:00+00   
40347 -75.559144  6.239717    686845  1.65192e+06  2018/12/01 00:00:00+00   

           hora  dia  periodo              

In [111]:
agg_df.isna().any()

x                 True
y                 True
objectid         False
radicado          True
fecha            False
hora             False
dia              False
periodo          False
clase             True
direccion        False
direccion_enc    False
cbml              True
tipo_geocod      False
gravedad         False
barrio            True
comuna            True
diseno            True
dia_nombre       False
mes              False
mes_nombre        True
x_magnamed       False
y_magnamed       False
longitud         False
latitud          False
file             False
shape             True
dtype: bool

In [112]:
agg_df[agg_df.radicado.isna()] # check the nulls

,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
39496,-75.561594,6.293715,546224,NaN,2017/12/21 00:00:00+00,09:58:00,21,2017,Atropello,CR 52 CL 100 - 75,CR 052 100 075 00000,02100090113,Nomenclatura con Interior,MUERTO,Santa Cruz,Santa Cruz,NaN,JUEVES,12,NaN,835743.14,1187952.15,-75.561594,6.293715,Accidentalidadgeorreferenciada2017.csv,NaN
40150,-75.578129,6.247262,546941,NaN,2017/09/06 00:00:00+00,05:45:00,6,2017,Atropello,CL 44 entre CR 59 y 60,"CL 044 59, 060 00000",1008,Malla vial,MUERTO,La Candelaria,Corazón de Jesús,NaN,MIÉRCOLES,9,NaN,833912.85,1182813.77,-75.578129,6.247262,Accidentalidadgeorreferenciada2017.csv,NaN
37075,-75.564198,6.261494,683301,NaN,2018/05/07 00:00:00+00,06:00:00,7,2018,Choque,CR 51 CL 64,CR 051 064 000 00000,1001,Malla vial,MUERTO,Prado,La Candelaria,NaN,LUNES,5,NaN,835454.94,1184388.09,-75.564198,6.261494,Accidentalidadgeorreferenciada2018.csv,NaN
37076,-75.546095,6.271341,683302,NaN,2018/11/01 00:00:00+00,07:40:00,1,2018,Choque,CL 80 CR 32 - 82,CL 080 032 082 00000,03040160010,Catastro con Interior,MUERTO,Santa Inés,Manrique,NaN,JUEVES,11,NaN,837458.56,1185477.35,-75.546095,6.271341,Accidentalidadgeorreferenciada2018.csv,NaN
37077,-75.593598,6.270184,683303,NaN,2018/12/26 00:00:00+00,12:30:00,26,2018,Choque,CL 59 CR 78 AA,CL 059 078 AA 000 00000,1114,Malla vial,MUERTO,Los Colores,Laureles Estadio,NaN,MIÉRCOLES,12,NaN,832200.75,1185349.38,-75.593598,6.270184,Accidentalidadgeorreferenciada2018.csv,NaN


In [113]:
agg_df.clase.value_counts(dropna=False)

Choque                170276
Otro                   26585
Atropello              24647
Caida Ocupante         16419
Volcamiento             8249
Caída de Ocupante       5665
Incendio                  32
Caída Ocupante            16
NaN                        6
Choque                     3
Choque y Atropello         1
Name: clase, dtype: int64

In [114]:
agg_df.cbml.value_counts(dropna=False)

1019           6182
0517           5246
1507           5153
1012           4988
1105           4461
               ... 
80004340062       1
14220240062       1
15100350002       1
02030120001       1
11140030016       1
Name: cbml, Length: 712, dtype: int64

In [115]:
agg_df.radicado.value_counts()[0:21]

62196                    3
1618580.0                2
39226                    2
1662064                  2
1640023.0                2
31435                    2
1656231                  2
1639346.0                2
1639954.0                2
58560                    2
1647440.0                2
58864                    2
1647286.0                2
1655403                  2
48996                    2
050016000206201523690    2
1640118.0                2
1640922.0                2
1640356.0                2
1638618.0                2
1689154                  1
Name: radicado, dtype: int64

In [116]:
agg_df[agg_df.radicado==1638618.0]

,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
21533,-75.585424,6.241112,666622,1.63862e+06,2018/08/14 00:00:00+00,05:35:00,14,2018,Choque,CR 66 CL 34 A,CR 066 034 A 000 00000,1105,Malla vial aproximada: CR 66-35,SOLO DAÑOS,Los Conquistadores,Laureles Estadio,Tramo de via,MARTES,8,NaN,833105.42,1182133.61,-75.585424,6.241112,Accidentalidadgeorreferenciada2018.csv,NaN
36260,-75.585424,6.241112,682426,1.63862e+06,2018/10/14 00:00:00+00,05:35:00,14,2018,Choque,CR 66 CL 34 A,CR 066 034 A 000 00000,1105,Malla vial aproximada: CR 66-35,SOLO DAÑOS,Los Conquistadores,Laureles Estadio,Tramo de via,DOMINGO,10,NaN,833105.42,1182133.61,-75.585424,6.241112,Accidentalidadgeorreferenciada2018.csv,NaN


In [117]:
df.radicado.value_counts(dropna=False)[0:30]

NaN          7
1687153.0    3
1687280.0    3
1689208.0    3
1687560.0    3
1687158.0    3
1687159.0    3
1687162.0    3
1687163.0    3
62196        3
1687164.0    3
1687165.0    3
1689064.0    3
1687166.0    3
1685159.0    3
1687167.0    3
1687169.0    3
1687154.0    3
1685140.0    3
1687171.0    3
1687292.0    3
1685517.0    3
1687136.0    3
1685493.0    3
1687138.0    3
1687557.0    3
1684827.0    3
1687140.0    3
1687144.0    3
1688636.0    3
Name: radicado, dtype: int64

In [118]:
df[df.radicado=="1638618"]

,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado
205150,2019-06-04T20:55:57.306Z,1105,A000846486,Malla vial aproximada: CR 66-35,CR 066 034 A 000 00000,Choque,"[-75.5854551105, 6.24105317866]",8.0,2018.0,Laureles Estadio,[u'incidentes'],CR 66 CL 34 A,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,11,Tramo de via,Los Conquistadores,Solo daños,2018-08-14T22:35:00.000Z,1182133.61,833105.42,1.0,211762.0,1638618
213113,2019-06-04T20:56:23.935Z,1105,A000846486,Malla vial aproximada: CR 66-35,CR 066 034 A 000 00000,Choque,"[-75.5854551105, 6.24105317866]",10.0,2018.0,Laureles Estadio,[u'incidentes'],CR 66 CL 34 A,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,11,Tramo de via,Los Conquistadores,Solo daños,2018-10-14T22:35:00.000Z,1182133.61,833105.42,1.0,219306.0,1638618


In [119]:
df[df.radicado=="1638617"]

,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado
203433,2019-06-04T20:55:58.641Z,402,A000846480,Malla vial,CR 051 092 000 00000,Choque,"[-75.5611117649, 6.28348983493]",8.0,2018.0,Aranjuez,[u'incidentes'],CR 51 CL 92,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,04,Tramo de via,San Isidro,Solo daños,2018-08-15T20:00:00.000Z,1186820.47,835813.52,1.0,211882.0,1638617


We found some duplicates values in "radicado" field so we procedure to drop them.

In [120]:
df = df.sort_values(by=["radicado", "id"])
df.drop_duplicates(subset=["radicado"], keep="first", inplace=True)

In order to have date and time in different columns we decided to derivate new columns from "fecha" and have separet columns for the date and the hour.

In [121]:
df["fecha_incidente"] = df.fecha.str[0:10]
df["hora_incidente"] = df.fecha.str[11:13]

In [122]:
# drop duplicates by "radicado", "objectid"
agg_df = agg_df.sort_values(by=["radicado", "objectid"])
agg_df.drop_duplicates(subset=["radicado"], keep="first", inplace=True)
agg_df = agg_df[(~agg_df.radicado.isna()) & (agg_df.radicado != "0")]

In [123]:
df.radicado

262153    1.67637e+06
262150    1.67644e+06
262168    1.67646e+06
262162    1.67646e+06
262160    1.67647e+06
             ...     
143894           9769
9909             9778
12090            9780
183864           9982
172418            NaN
Name: radicado, Length: 261876, dtype: object

In [124]:
df[df.radicado.isna()] # check the number of nulls

,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado,fecha_incidente,hora_incidente
172418,2019-06-04T20:54:26.357Z,1008,A000665813,Malla vial,"CL 044 59, 060 00000",Atropello,"[-75.5781789259, 6.24722232125]",9.0,2017.0,La Candelaria,[u'incidentes'],CL 44 entre CR 59 y 60,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,10,NaN,Corazón de Jesús,Con muertos,2017-09-06T10:45:00.000Z,1182813.77,833912.85,1.0,185160.0,NaN,2017-09-06,10


In [125]:
agg_df = agg_df[~agg_df.radicado.isna()]
df=df[~df.radicado.isna()]
dtype = dict(radicado=str)
agg_df.radicado = agg_df.radicado.astype(dtype)
df.radicado = df.radicado.astype(dtype)
agg_df.radicado = agg_df.radicado.str.strip()
agg_df.radicado = agg_df.radicado.str.replace("(\d+)(.0)*","\\1") #.dtype #.sort_values(by="radicado", ascending=False)
agg_df.radicado = agg_df.radicado.str.replace("^(0+)(\d+)", "\\2")
df.radicado = df.radicado.str.strip()
df.radicado = df.radicado.str.replace("(\d+)(.0)*","\\1")

In [126]:
df.fecha.max()

'2019-10-01T04:20:00.000Z'

In [131]:
df.shape
df
# r = df[(df.cbml.isna()) & (df.x_long_y_lat == '[-75.7037762763, 6.22141524356]') ][['x_long_y_lat','barrio','comuna','direccion','direccion_enc','cbml']]
# r

,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado,fecha_incidente,hora_incidente
262153,2019-11-07T16:07:39.311Z,103,A001002056,Malla vial,CR 043 A 110 B 000 00000,Atropello,"[-75.5478922382, 6.29966416516]",6.0,2019.0,Popular,[u'incidentes'],CR 43 A CL 110 B,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,01,Tramo de via,Popular,Con heridos,2019-07-01T02:50:00.000Z,1.188606e+06,837281.840000,1.0,10647.0,1676369,2019-07-01,02
262150,2019-11-07T16:07:39.311Z,1102,A001002214,Malla vial,CR 066 048 D 000 00000,Choque,"[-75.5829966564, 6.25459513351]",6.0,2019.0,Laureles Estadio,[u'incidentes'],CR 66 CL 48 D,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,11,Lote o Predio,Suramericana,Solo daños,2019-07-01T01:40:00.000Z,1.183631e+06,833381.860000,1.0,10641.0,1676437,2019-07-01,01
262168,2019-11-07T16:07:39.312Z,607,A001002310,Malla vial,CR 083 091 A 000 00000,Choque,"[-75.5849391248, 6.28861482868]",7.0,2019.0,Doce de Octubre,[u'incidentes'],CR 83 CL 91 A,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,06,Tramo de via,Kennedy,Solo daños,2019-07-01T23:20:00.000Z,1.187395e+06,833177.650000,1.0,10699.0,1676456,2019-07-01,23
262162,2019-11-07T16:07:39.312Z,312,A001002302,Malla vial,CR 023 B 084 B 000 00000,Choque,"[-75.5378038865, 6.27686386215]",7.0,2019.0,Manrique,[u'incidentes'],CR 23 B CL 84 B,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,03,Tramo de via,Oriente,Solo daños,2019-07-01T20:30:00.000Z,1.186080e+06,838391.450000,1.0,10686.0,1676461,2019-07-01,20
262160,2019-11-07T16:07:39.312Z,NaN,A001002169,No Ubicada,CR 067 025 000 00000,Choque,"[-75.7037762763, 6.22141524356]",7.0,2019.0,Sin Inf,[u'incidentes'],CR 67 CL 25,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,No ubicada,Tramo de via,NaN,Con heridos,2019-07-01T16:40:00.000Z,1.180000e+06,820000.000000,1.0,10674.0,1676468,2019-07-01,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12089,2019-06-04T20:44:08.013Z,3.02021e+09,A1485619,Catastro con Interior,CR 039 085 A 035 00000,Choque,"[-75.5477362391, 6.27745474338]",2.0,2014.0,Manrique,[u'incidentes'],CR 39 CL 85 A - 35,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,03,NaN,Las Granjas,Con muertos,2014-02-24T08:03:00.000Z,1.186149e+06,837292.200000,1.0,6512.0,9766,2014-02-24,08
143894,2019-06-04T20:53:18.109Z,6.08101e+09,A000497560,Catastro con Interior,CR 083 101 A 021 00000,Atropello,"[-75.5821690996, 6.30114194037]",2.0,2017.0,Doce de Octubre,[u'incidentes'],CR 83 CL 101 A - 21,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,06,NaN,Picacho,Con muertos,2017-02-12T01:30:00.000Z,1.188780e+06,833488.260000,1.0,165498.0,9769,2017-02-12,01
9909,2019-06-04T20:44:08.014Z,1.50304e+10,A1485792,Catastro con Interior,CL 025 065 C 050 00000,Atropello,"[-75.5855316677, 6.22630141567]",2.0,2014.0,Guayabal,[u'incidentes'],CL 25 CR 65 C - 50,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,15,NaN,Trinidad,Con muertos,2014-02-24T12:30:00.000Z,1.180502e+06,833092.280000,1.0,6513.0,9778,2014-02-24,12
12090,2019-06-04T20:44:08.014Z,1.62002e+10,A1485790,Nomenclatura con Interior,CL 033 078 177 00000,Atropello,"[-75.6000288989, 6.23861225192]",2.0,2014.0,Belén,[u'incidentes'],CL 33 CR 78 - 177,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,16,NaN,El Nogal-Los Almendros,Con muertos,2014-02-24T12:15:00.000Z,1.181868e+06,831491.290000,1.0,6515.0,9780,2014-02-24,12


In [128]:
df.clase = df.clase.apply(lambda x: unidecode(x).lower().strip().replace(' de ',' '))

AttributeError: 'float' object has no attribute 'encode'

In [129]:
df.to_csv("../processed/preprocessed_accidents.csv",index=False)